<a href="https://colab.research.google.com/github/akashsharma-2002/twitter-scrapper/blob/main/twitter_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import requests
from bs4 import BeautifulSoup

# Scrape proxy IP addresses from the URL
def scrape_proxy_ips(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', class_='table-striped')
    proxies = []
    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        ip = columns[0].text.strip()
        port = columns[1].text.strip()
        protocol = columns[6].text.strip().lower()
        if protocol == 'yes':
            protocol = 'https'
        else:
            protocol = 'http'
        proxies.append(f"{protocol}://{ip}:{port}")
    return proxies

# Function to scrape Twitter profile details using rotating proxy IPs
def scrape_twitter_profile(username, proxies):
    url = f"https://twitter.com/{username}"

    response = None
    for proxy in proxies:
        try:
            response = requests.get(url, proxies={"http": proxy, "https": proxy}, timeout=5)
            response.raise_for_status()
            break  # Break the loop if successful response
        except requests.exceptions.RequestException:
            continue  # Try the next proxy

    if response:
        try:
            soup = BeautifulSoup(response.content, 'html.parser')

            biography = soup.find('div', class_='ProfileHeaderCard-bio u-dir').text.strip()

            followers_elem = soup.find('li', {'data-nav': 'followers'}).find('span', class_='ProfileNav-value')
            followers_count = int(followers_elem.get('data-count', '0').replace(',', ''))

            following_elem = soup.find('li', {'data-nav': 'following'}).find('span', class_='ProfileNav-value')
            following_count = int(following_elem.get('data-count', '0').replace(',', ''))

            likes_elem = soup.find('li', {'data-nav': 'favorites'}).find('span', class_='ProfileNav-value')
            likes_count = int(likes_elem.get('data-count', '0').replace(',', ''))

            user_id_elem = soup.find('div', class_='ProfileNav').find('a', class_='ProfileNav-stat ProfileNav-stat--link')
            user_id = int(user_id_elem['href'].split('/')[-1]) if user_id_elem else 0

            return {
                'biography': biography,
                'followers_count': followers_count,
                'following_count': following_count,
                'likes_count': likes_count,
                'user_id': user_id
            }
        except AttributeError:
            print("Failed to extract the profile data.")
            return None
    else:
        print("Failed to fetch the Twitter profile.")
        return None

# Scrape proxy IPs from the provided URL
proxy_url = "https://free-proxy-list.net/"
proxies = scrape_proxy_ips(proxy_url)

# Define the target Twitter profile username
username = "sachin_rt"

# Scrape Twitter profile details using rotating proxy IPs
profile_data = scrape_twitter_profile(username, proxies)
if profile_data:
    print(profile_data)


Failed to extract the profile data.
